In [1]:
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
share = {}

In [3]:
# model_name = "falcon-40b-instruct"
# model_path = f"/gpfs/user/jsh/infer-large/{model_name}"

# share["tokenizer"] = AutoTokenizer.from_pretrained(model_path)
# share["model"] = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     device_map="auto",
#     torch_dtype=torch.float16,
#     trust_remote_code=True,
# )

In [4]:
model_name = "falcon-7b-instruct"
model_path = f"/gpfs/user/jsh/infer/{model_name}"

share["tokenizer"] = AutoTokenizer.from_pretrained(model_path)
share["model"] = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).cuda(7)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
memories = [torch.cuda.memory_reserved(i) / 1024 ** 3 for i in range(8)]
print(f"{sum(memories)} GiB")

25.916015625 GiB


In [6]:
prompt = ["What is deep learning?"]
inputs = share["tokenizer"](prompt, return_tensors="pt").input_ids.cuda(7)
inputs

tensor([[1562,  304, 2845, 2860,   42]], device='cuda:7')

In [7]:
total_memory = 0
outputs = share["model"].generate(
    inputs,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    min_length=20,
    max_length=200,
    temperature=0.9,
    repetition_penalty=1.5,
    no_repeat_ngram_size=3,
)

sentences = share["tokenizer"].batch_decode(outputs, skip_special_tokens=True)
memories = [torch.cuda.memory_reserved(i) / 1024 ** 3 for i in range(8)]
print(f"{sum(memories)} GiB")

/home/jsh/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


26.2734375 GiB


In [8]:
torch.cuda.empty_cache()
memories = [torch.cuda.memory_reserved(i) / 1024 ** 3 for i in range(8)]
print(f"{sum(memories)} GiB")

25.951171875 GiB


In [9]:
share["model"].cpu()
share.clear()
gc.collect()
torch.cuda.empty_cache()
memories = [torch.cuda.memory_reserved(i) / 1024 ** 3 for i in range(8)]
print(f"{sum(memories)} GiB")

0.03515625 GiB


In [1]:
import requests

port = 1040

In [2]:
model_name = "falcon-40b-instruct"

In [3]:
url = f"http://127.0.0.1:{port}/load-model-large"
rs = requests.post(url, json={"name": model_name})
rs.json()

{'elapsed_time': 84.11809825897217, 'used_memory': 77.9375}

In [4]:
url = f"http://127.0.0.1:{port}/infer-large"
rs = requests.post(url, json={"sentences": ["What is deep learning?"]})
rs.json()

{'elapsed_time': 17.167015552520752, 'used_memory': 78.693359375}

In [5]:
url = f"http://127.0.0.1:{port}/unload-model-large"
rs = requests.post(url, json={"name": model_name})
rs.json()

{'elapsed_time': 43.24610185623169, 'used_memory': 1.127773430198431e-09}

In [6]:
url = f"http://127.0.0.1:{port}/load-model-large"
rs = requests.post(url, json={"name": model_name})
rs.json()

{'elapsed_time': 70.33183932304382, 'used_memory': 78.24218749887223}

In [7]:
url = f"http://127.0.0.1:{port}/infer-large"
rs = requests.post(url, json={"sentences": ["What is deep learning?"]})
rs.json()

{'elapsed_time': 6.915565013885498, 'used_memory': 79.34960937387223}

In [8]:
url = f"http://127.0.0.1:{port}/unload-model-large"
rs = requests.post(url, json={"name": model_name})
rs.json()

{'elapsed_time': 44.58307361602783, 'used_memory': 1.127773430198431e-09}

In [9]:
model_name = "llama-30b-instruct"

In [10]:
url = f"http://127.0.0.1:{port}/load-model-large"
rs = requests.post(url, json={"name": model_name})
rs.json()

{'elapsed_time': 56.409790992736816, 'used_memory': 61.35742187387223}

In [11]:
url = f"http://127.0.0.1:{port}/infer-large"
rs = requests.post(url, json={"sentences": ["What is deep learning?"]})
rs.json()

{'elapsed_time': 6.706385850906372, 'used_memory': 61.73046874887223}

In [12]:
url = f"http://127.0.0.1:{port}/unload-model-large"
rs = requests.post(url, json={"name": model_name})
rs.json()

{'elapsed_time': 35.39140582084656, 'used_memory': 1.127773430198431e-09}